In [1]:
import pandas as pd
import torch
import numpy as np
import pickle

## Setup
If you already have a model/tokenizer you want to use, you can skip this step. 
Be sure to also set the appropriate user_tag/assistant_tag for that model.

In [2]:
%%capture
# The quantized model used here requires some extra libraries. 
import sys
!{sys.executable} -m pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu121
!{sys.executable} -m pip install optimum>=1.12.0
!{sys.executable} -m pip install auto-gptq==0.6.0
!{sys.executable} -m pip install accelerate

In [3]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/workspace/cache/' # change or comment out as desired 
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model(model_name_or_path, revision, device):
    model = AutoModelForCausalLM.from_pretrained(
        model_name_or_path, device_map=device, revision=revision, trust_remote_code=False)
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True, padding_side="left")
    tokenizer.pad_token_id = 0
    return model, tokenizer

device = 'cuda:0'
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
revision = 'gptq-4bit-32g-actorder_True'
user_tag = "[INST] "
assistant_tag = " [/INST]"

model, tokenizer = load_model(model_name_or_path, revision, device)

/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` in

## Generate responses with model

In [2]:
# Get the GPT-4 prompts
content_prompts = pd.read_csv('./creative_content/content_prompts_gpt4.csv', header=None)
content_prompts.columns = ['prompt', 'label']

In [7]:
content_prompts.head()

,prompt,label
0,Write a sci-fi short story about a robot exper...,1
1,Compose a mystery novel opening about a detect...,1
2,Develop a screenplay scene where two estranged...,1
3,Pen a children's story about a mischievous kit...,1
4,Craft a poem celebrating the beauty of a snowy...,1


In [34]:
from lmdoctor.utils import format_prompt

gen_only=True
prompts = content_prompts['prompt'].tolist()
batch_size=10
all_texts = []

for i in range(0, len(prompts), batch_size):

    prompts_batch = prompts[i:i+batch_size]

    formatted_prompts = []
    for prompt in prompts_batch:
        formatted_prompt = format_prompt(prompt, user_tag, assistant_tag)
        formatted_prompts.append(formatted_prompt)
    
    model_inputs = tokenizer(formatted_prompts, return_tensors='pt', padding=True).to(device)
    
    with torch.no_grad():
        sequences = model.generate(**model_inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=30)
    
    start_gen_idx = model_inputs.input_ids.shape[1]
    sequences = sequences[:, start_gen_idx:] if gen_only else sequences
    these_texts = tokenizer.batch_decode(sequences, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    all_texts.extend(these_texts)

In [42]:
content_prompts['response'] = all_texts
content_prompts.head(2)

,prompt,label,response
0,Write a sci-fi short story about a robot exper...,1,"In the vast expanse of the cosmos, nestled bet..."
1,Compose a mystery novel opening about a detect...,1,Chapter 1: The Map's Enigma\n\nIn the heart of...


In [44]:
content_prompts.to_csv('./creative_content/content_prompts_with_response.csv', index=False)